In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
with open('X_train.pickle', 'rb') as f:
    X_train = pickle.load(f)
with open('y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)

In [8]:
print('X Train lenght: ', len(X_train))
print('Y Train length: ', len(y_train))

X Train lenght:  918
Y Train length:  393


In [9]:
rec_df = pd.read_csv('Rec_face.csv')

In [13]:
X_test = rec_df['Recognizable-Facial-Feature'][:len(X_train)]
y_test = rec_df['Recognizable-Facial-Feature'][:len(y_train)]